In [182]:
import pandas as pd
import numpy as np
import requests as req
import requests as req
from bs4 import BeautifulSoup

df=pd.read_csv('gtd.csv', encoding="ISO-8859-1")
df1 = df.drop(columns=["Unnamed: 0","vicinity"])
df2= df1.rename(columns={'iyear':'Year','imonth':'Month','country_txt':'NCountry','region_txt':'Region','nkill':'Killed'},inplace=True)
#mergeado2["dead"] = mergeado[mergeado["Fatal"]=="Y"].groupby(["Country", "Code"]).count()["Fatal"]
df1.head(20)

,Year,Month,extended,country,NCountry,region,Region,provstate,latitude,longitude,attacktype1_txt,gname,weaptype1_txt,Killed,nwound
0,1970,7,0,58,Dominican Republic,2,Central America & Caribbean,NaN,18.456792,-69.951164,Assassination,MANO-D,Unknown,1.0,0.0
1,1970,0,0,130,Mexico,1,North America,Federal,19.371887,-99.086624,Hostage Taking (Kidnapping),23rd of September Communist League,Unknown,0.0,0.0
2,1970,1,0,160,Philippines,5,Southeast Asia,Tarlac,15.478598,120.599741,Assassination,Unknown,Unknown,1.0,0.0
3,1970,1,0,78,Greece,8,Western Europe,Attica,37.997490,23.762728,Bombing/Explosion,Unknown,Explosives,NaN,NaN
4,1970,1,0,101,Japan,4,East Asia,Fukouka,33.580412,130.396361,Facility/Infrastructure Attack,Unknown,Incendiary,NaN,NaN
5,1970,1,0,217,United States,1,North America,Illinois,37.005105,-89.176269,Armed Assault,Black Nationalists,Firearms,0.0,0.0
6,1970,1,0,218,Uruguay,3,South America,Montevideo,-34.891151,-56.187214,Assassination,Tupamaros (Uruguay),Firearms,0.0,0.0
7,1970,1,0,217,United States,1,North America,California,37.791927,-122.225906,Bombing/Explosion,Unknown,Explosives,0.0,0.0
8,1970,1,0,217,United States,1,North America,Wisconsin,43.076592,-89.412488,Facility/Infrastructure Attack,New Year's Gang,Incendiary,0.0,0.0
9,1970,1,0,217,United States,1,North America,Wisconsin,43.072950,-89.386694,Facility/Infrastructure Attack,New Year's Gang,Incendiary,0.0,0.0


In [135]:
# This is the url you will scrape in this exercise
url = 'https://countrycode.org'

In [172]:

def scrapear(url, css):
    res = req.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    ss = soup.select(css)
    l=list(map(lambda x : x.text.strip(),ss))
    return l
pais =scrapear(url, "tr td a")
eco=scrapear(url, "tr td")[3::6]
ec=eco[:240]
paises= pais[:240]
pas=scrapear(url, "tr td")[5::6]
pasta=pas[:240]
pasta
pai=pd.DataFrame()

pai["paises"]= paises
pai["pob"]= ec
pai["pasta"]= pasta

pai.head()




,paises,pob,pasta
0,Afghanistan,"29,121,286",20.65 Billion
1,Albania,"2,986,952",12.8 Billion
2,Algeria,"34,586,184",215.7 Billion
3,American Samoa,"57,881",462.2 Million
4,Andorra,"84,000",4.8 Billion


In [175]:
merged = pai.merge(df1, left_on = 'paises', right_on = 'NCountry')
merged.head()

,paises,pob,pasta,Year,Month,extended,country,NCountry,region,Region,provstate,latitude,longitude,attacktype1_txt,gname,weaptype1_txt,Killed,nwound
0,Afghanistan,"29,121,286",20.65 Billion,1973,5,0,4,Afghanistan,6,South Asia,Kabul,34.516895,69.147011,Unknown,Black December,Unknown,0.0,1.0
1,Afghanistan,"29,121,286",20.65 Billion,1979,2,0,4,Afghanistan,6,South Asia,Kabul,34.516895,69.147011,Hostage Taking (Kidnapping),Shia Muslim extremists,Unknown,1.0,0.0
2,Afghanistan,"29,121,286",20.65 Billion,1979,8,0,4,Afghanistan,6,South Asia,Ghazni,33.542622,68.415329,Bombing/Explosion,Muslim Guerrillas,Explosives,50.0,0.0
3,Afghanistan,"29,121,286",20.65 Billion,1979,9,0,4,Afghanistan,6,South Asia,Herat,34.346722,62.197315,Armed Assault,Muslim Guerrillas,Firearms,2.0,1.0
4,Afghanistan,"29,121,286",20.65 Billion,1987,5,0,4,Afghanistan,6,South Asia,Unknown,NaN,NaN,Unarmed Assault,Afghan Mujahideen,Chemical,0.0,2.0


In [177]:
merged.gname.value_counts()

Unknown                                             81325
Taliban                                              7478
Islamic State of Iraq and the Levant (ISIL)          5610
Shining Path (SL)                                    4555
Farabundo Marti National Liberation Front (FMLN)     3351
Al-Shabaab                                           3288
New People's Army (NPA)                              2772
Irish Republican Army (IRA)                          2643
Revolutionary Armed Forces of Colombia (FARC)        2487
Boko Haram                                           2418
Kurdistan Workers' Party (PKK)                       2310
Basque Fatherland and Freedom (ETA)                  2024
Communist Party of India - Maoist (CPI-Maoist)       1878
Maoists                                              1630
Liberation Tigers of Tamil Eelam (LTTE)              1606
National Liberation Army of Colombia (ELN)           1561
Tehrik-i-Taliban Pakistan (TTP)                      1351
Houthi extremi

In [ ]:
#holaaaa